In [8]:
import pandas as pd
pd.set_option("display.max_rows", 1000)
from datetime import datetime, date
import requests
import json

# pull in felony intake data
url="https://datacatalog.cookcountyil.gov/resource/3k7z-hchi.json?$limit=1000000"

request = requests.get(url)
data_dict = json.loads(request.text)

felony_intakes = pd.DataFrame(data_dict)

felony_intakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429724 entries, 0 to 429723
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   case_id                  429724 non-null  object
 1   case_participant_id      429724 non-null  object
 2   received_date            429724 non-null  object
 3   offense_category         429724 non-null  object
 4   participant_status       407297 non-null  object
 5   age_at_incident          407695 non-null  object
 6   race                     416498 non-null  object
 7   gender                   418512 non-null  object
 8   incident_begin_date      411058 non-null  object
 9   law_enforcement_agency   412059 non-null  object
 10  arrest_date              389894 non-null  object
 11  felony_review_date       251224 non-null  object
 12  felony_review_result     251224 non-null  object
 13  update_offense_category  429724 non-null  object
 14  incident_city       

In [61]:
# all of the date columns in here are not date types, let's fix that
intake_cols = felony_intakes.columns

for col in intake_cols:
    if '_date' in col:
        felony_intakes[col] = pd.to_datetime(felony_intakes[col], errors='coerce')
    
# get year from created date
felony_intakes['year'] = felony_intakes['received_date'].dt.year

In [62]:
# What are the unique values in some of these categorical columns
unique_offense_cat = felony_intakes['offense_category'].unique()
unique_participant_status = felony_intakes['participant_status'].unique()
unique_enforcement_agency = felony_intakes['law_enforcement_agency'].unique()
unique_review_result = felony_intakes['felony_review_result'].unique()
unique_updated_offense = felony_intakes['update_offense_category'].unique()

In [63]:
# how does this data shake up if we restrict it to only Chicago
chicago_felonies = felony_intakes[felony_intakes['incident_city']=='Chicago']
non_chicago_felonies = felony_intakes[felony_intakes['incident_city']!='Chicago']

# Uncomment to see info
chicago_felonies.info()
#non_chicago_felonies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295042 entries, 7 to 429723
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   case_id                  295042 non-null  object        
 1   case_participant_id      295042 non-null  object        
 2   received_date            295042 non-null  datetime64[ns]
 3   offense_category         295042 non-null  object        
 4   participant_status       282718 non-null  object        
 5   age_at_incident          292078 non-null  object        
 6   race                     288915 non-null  object        
 7   gender                   289602 non-null  object        
 8   incident_begin_date      294474 non-null  datetime64[ns]
 9   law_enforcement_agency   295042 non-null  object        
 10  arrest_date              273218 non-null  datetime64[ns]
 11  felony_review_date       157178 non-null  datetime64[ns]
 12  felony_review_re

In [64]:
# in chicago, what units are charging the most felonies?
chicago_unit_felonies = chicago_felonies.groupby('unit').agg('size').sort_values(ascending=False)
#chicago_unit_felonies

In [65]:
# in chicago, why are some of the 'unit' fields null?
null_units = chicago_felonies[chicago_felonies['unit'].isna()].reset_index()

# Is it by date?
#print("Null Units by Year")
#print(null_units.groupby('year').agg('size'), "\n")

# Is it by status?
#print("Null Units by Status")
#print(null_units.groupby('participant_status').agg('size').sort_values(ascending=False), "\n")

# Is it by enforcement agency?
#print("Null Units by Enforcement Agency")
#print(null_units.groupby('law_enforcement_agency').agg('size').sort_values(ascending=False), "\n")

# Is it by review result?
#print("Null Units by Review Result")
#print(null_units.groupby('felony_review_result').agg('size').sort_values(ascending=False), "\n")

# Is it by review result?
#print("Null Units by Updated Offense")
#print(null_units.groupby('update_offense_category').agg('size').sort_values(ascending=False), "\n")


### There's about 14,000 arrests that were made in Chicago BUT NOT by CPD. Therefore, unit does not likely apply. 
### Still, null unit rate is about 40% in Chicago arrests

In [79]:
# what are the unique review results?
print("Unique Review Results\n",unique_review_result)

# subset down to just 'Approved', 'Charges(S) Approved', 'Rejected', & 'Disregard'
statuses = ['Approved','Charge(S) Approved', 'Arrest Warrant', 'Search Warrant','Rejected','Disregard']
rejections = ['Rejected','Disregard']

chicago_felonies[chicago_felonies['felony_review_result'].isin(statuses)]\
        .groupby(['felony_review_result']).agg('size')

chicago_felonies[chicago_felonies['felony_review_result'].isin(rejections)]\
        .groupby(['year', 'unit']).agg('size').sort_values(ascending=False)\
        .sort_index(level='year', sort_remaining=False)

Unique Review Results
 ['Approved' 'Charge(S) Approved' nan 'Felony Review' 'Rejected'
 'Continued Investigation' 'Search Warrant' 'Victim Sensitive Interview'
 'Advice' 'Screen Felony' 'VICTIM SENSITIVE INT' 'Disregard'
 'Warr Ord Warr Issued' 'Arrest Warrant' 'Deputy Override' 'Warrant Exec'
 'Chief Judge Assignment To Felony Courtroom']


year  unit                                                     
2011  UNIT 079 - SPECIAL INVESTIGATIONS                            64
      District 6 - Gresham                                         46
      District 8 - Chicago Lawn                                    35
      District 3 - Grand Crossing                                  31
      District 7 - Englewood                                       28
      District 25 - Grand Central                                  25
      District 5 - Calumet                                         25
      District 11 - Harrison                                       20
      District 15 - Austin                                         20
      District 9 - Deering                                         20
      District 1 - Central                                         19
      UNIT 932 - PATROL AREA 2                                     17
      District 12 - Monroe                                         17
      District 4 - South C